# Systematicity in English monomorphemic words by word class

### Sean Trott

Do certain word classes have more sub-morphemic systematicity than others?

## Load model and dataset

In [1]:
import os 
import gensim
import numpy as np
import pandas as pd
import re

# Variables
MODEL_PATH = os.environ['WORD2VEC_PATH']
ROOT_PATH = 'data/raw/roots_celex_monosyllabic.txt'

Using TensorFlow backend.


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format(MODEL_PATH, binary=True)

In [3]:
entries = open(ROOT_PATH, "r").read().split("\n")

In [4]:
words = [entry.split("\\")[0] for entry in entries if entry != "" and entry.islower()]
words[0]

'a'

## Filter by words that appear in model

In [5]:
critical_words = list(set([w for w in words if w in model.vocab]))

In [6]:
len(critical_words)

2082

## Obtain form and meaning similarity metrics

Here, we import the class `SystematicityUtilities` from a [custom library](https://github.com/seantrott/nlp_utilities). By default, this class uses *Levenshtein distance* as its metric for *form similarity*, and *cosine distance* as its metric for *meaning similarity*. The `compare_form_and_meaning` method used below compares every word pair along form and meaning dimensions.

In [7]:
from nlp_utilities.compling import SystematicityUtilities
systematicity_utils = SystematicityUtilities(model)
comparisons = systematicity_utils.compare_form_and_meaning(critical_words)

In [8]:
import pandas as pd

In [9]:
comparisons_df = pd.DataFrame.from_dict(comparisons)

In [11]:
print("{length} comparisons total".format(length=len(comparisons_df)))

2166321 comparisons total


In [10]:
comparisons_df.sample(4)

,form,meaning,w1,w2
354126,5,0.078198,voile,bleat
1190297,4,-0.000454,taint,meet
1928647,4,0.070623,oust,jam
27053,3,0.087516,vice,ti


## Global correlation

In [12]:
from scipy.stats import linregress

In [14]:
true_regression = linregress(comparisons_df['form'], comparisons_df['meaning'])
true_regression.rvalue

-0.040672612879521897

In other words, words with more **form differences** (e.g. a higher Levenshtein distance) will have *less* similar meanings (e.g. cosine similarity).

## Compare global correlation to permuted distributions

In [16]:
import numpy as np

In [ ]:
permuted_results = []
for permute in range(100):
    print(permute)
    permuted_meaning = np.random.permutation(comparisons_df['meaning'])
    random_regression = linregress(comparisons_df['form'], permuted_meaning)
    permuted_results.append(random_regression)

In [ ]:
s = [reg.rvalue for reg in permuted_results]
s